In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from collections import Counter


In [2]:
fn = "Erklärung Gemeinden Jahr 2020 Tramin.xlsx"

In [3]:
df = pd.read_excel(fn)

In [4]:
    df.columns = [
        "Numero \nprogressivo",
        "Cognome e nome bambino",
        "Data di nascita",
        "Codice fiscale",
        "Assistente domiciliare all'infanzia",
        "Comune di residenza assistente domiciliare all'infanzia",
        "Data inizio contratto (o data inizio assistenza se diversa)",
        "Data fine contratto\n(o data fine assistenza se diversa) *",
        "Ore di assistenza \n ai sensi della delibera\nn. 666/2019",
        "Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020",
        "Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020",
        "Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo)",
        "Ore totali rendicontate per il 2020",
    ]

In [5]:
traeger = df.iloc[2]

In [6]:
type(traeger[4]) == type("string")

False

In [7]:
traeger = traeger[3].title()
gemeinde = df.iloc[3]
gemeinde = gemeinde[3]
gemeinde

'Tramin'

In [8]:
df = df.drop(labels=range(0, 8), axis=0)
cond = df["Numero \nprogressivo"] == "*"
df.loc[cond, "Numero \nprogressivo"] = 99
df.loc[cond, "Cognome e nome bambino"] += " *"
df["Numero \nprogressivo"] = pd.to_numeric(
    df["Numero \nprogressivo"], errors="coerce"
)
validi = df["Numero \nprogressivo"].notna()
df = df[validi]

In [9]:
condizione = pd.isnull(
    df["Data fine contratto\n(o data fine assistenza se diversa) *"]
)
anno_riferimento = 2020
df.loc[
    condizione, "Data fine contratto\n(o data fine assistenza se diversa) *"
] = "12-31-" + str(anno_riferimento)

anno_riferimento = 2020
df.loc[
    condizione, "Data fine contratto\n(o data fine assistenza se diversa) *"
] = "12-31-" + str(anno_riferimento)
df["Data di nascita"] = df["Data di nascita"].astype("datetime64[ns]")

df["Data fine contratto\n(o data fine assistenza se diversa) *"] = df[
    "Data fine contratto\n(o data fine assistenza se diversa) *"
].astype("datetime64[ns]")

df["Data inizio contratto (o data inizio assistenza se diversa)"] = df[
    "Data inizio contratto (o data inizio assistenza se diversa)"
].astype("datetime64[ns]")

df["Numero \nprogressivo"] = pd.to_numeric(
    df["Numero \nprogressivo"], errors="coerce"
)

df = df.drop(["Numero \nprogressivo"], axis=1)

df.insert(1, "Comune", gemeinde)
df.insert(1, "Ente", traeger)
df = df.fillna(0)
df["Codice fiscale"] = df["Codice fiscale"].str.upper()

ar = 2020

df["inizioNorm"] = df[
    "Data inizio contratto (o data inizio assistenza se diversa)"
]

df["fineNorm"] = df["Data fine contratto\n(o data fine assistenza se diversa) *"]

iniz = df["inizioNorm"].dt.year < ar

df.loc[iniz, "inizioNorm"] = "01-01-" + str(
    ar
) 

fin = df["fineNorm"].dt.year > ar

df.loc[fin, "fineNorm"] = "12-31-" + str(ar)

df.insert(
    9, "GiorniAssistenzaAnnoRiferimento", 0
) 

df["GiorniAssistenzaAnnoRiferimento"] = (
    df["fineNorm"] - df["inizioNorm"]
).dt.days + 1

In [11]:
condizioneerrore = ((1920 * (df["GiorniAssistenzaAnnoRiferimento"])) / 366) < df[
    "Ore totali rendicontate per il 2020"
]
    
condizioneerrore2 = ((1920 * (df.groupby("Codice fiscale")["GiorniAssistenzaAnnoRiferimento"].transform("sum")) / 366)) < df.groupby("Codice fiscale")["Ore totali rendicontate per il 2020"].transform("sum")

In [29]:
for i in df[~codfisc1]["Codice fiscale"]:
    print(i)

KJNMIA18M43A952A
KJNMIA18M43A952A
TMGMRM19B41A952N
TMGMRM19B41A952N
TMGMRM19B41A952N
TMGMRM19B41A952N
VNCDLN19B19A952G
VNCDLN19B19A952G
ZLGSML18E08F132P
ZLGSML18E08F132P
ZLGSML18E08F132P
ZLGSML18E08F132P
ZLGSML18E08F132P
ZLGSML18E08F132P
ZLGSML18E08F132P


In [19]:
codfisc1 = df.groupby("Codice fiscale")["Codice fiscale"].transform("count") == 1

In [27]:
df[condizioneerrore2 & ~codfisc1]

,Cognome e nome bambino,Ente,Comune,Data di nascita,Codice fiscale,Assistente domiciliare all'infanzia,Comune di residenza assistente domiciliare all'infanzia,Data inizio contratto (o data inizio assistenza se diversa),Data fine contratto\n(o data fine assistenza se diversa) *,GiorniAssistenzaAnnoRiferimento,Ore di assistenza \n ai sensi della delibera\nn. 666/2019,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020,Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo),Ore totali rendicontate per il 2020,inizioNorm,fineNorm
12,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Ambach Maria,Eppan,2020-08-17,2020-08-28,12,71.50,0.00,0,0.0,71.50,2020-08-17,2020-08-28
13,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Huber Melanie,Margreid,2020-11-25,2020-11-27,3,26.50,0.00,0,0.0,26.50,2020-11-25,2020-11-27
14,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Peer Petra,Kurtatsch,2020-01-02,2020-05-06,126,189.75,270.50,0,332.5,792.75,2020-01-02,2020-05-06
15,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Peer Petra,Kurtatsch,2020-07-20,2020-12-31,165,804.00,10.25,0,0.0,814.25,2020-07-20,2020-12-31


In [48]:
ll = ["Eppan","Bozen"]

In [49]:
ddff = df["Comune di residenza assistente domiciliare all'infanzia"].str.contains("Epp") | df["Comune di residenza assistente domiciliare all'infanzia"].str.contains("Boz")

In [50]:
df[ddff]

,Cognome e nome bambino,Ente,Comune,Data di nascita,Codice fiscale,Assistente domiciliare all'infanzia,Comune di residenza assistente domiciliare all'infanzia,Data inizio contratto (o data inizio assistenza se diversa),Data fine contratto\n(o data fine assistenza se diversa) *,GiorniAssistenzaAnnoRiferimento,Ore di assistenza \n ai sensi della delibera\nn. 666/2019,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020,Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo),Ore totali rendicontate per il 2020,inizioNorm,fineNorm
8,Chirone Riccardo,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-10-09,CHRRCR18R09E986O,Salvadori Paola,Bozen,2019-11-19,2020-12-31,366,1308.75,89.50,0,-340.00,1058.25,2020-01-01,2020-12-31
11,Kajana Mia,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-08-03,KJNMIA18M43A952A,Ambach Maria,Eppan,2019-08-06,2020-05-04,125,99.50,144.75,0,66.25,310.50,2020-01-01,2020-05-04
12,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Ambach Maria,Eppan,2020-08-17,2020-08-28,12,71.50,0.00,0,0.00,71.50,2020-08-17,2020-08-28
16,Von Call Dylan,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-19,VNCDLN19B19A952G,Ambach Maria,Eppan,2020-07-20,2020-11-23,127,231.50,20.75,0,0.00,252.25,2020-07-20,2020-11-23
17,Von Call Dylan,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-19,VNCDLN19B19A952G,Ambach Maria,Eppan,2019-11-25,2020-05-06,127,173.50,279.50,0,364.50,817.50,2020-01-01,2020-05-06
21,Zelger Samuel,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-05-08,ZLGSML18E08F132P,Ambach Maria,Eppan,2020-09-04,2020-09-04,1,8.00,0.00,0,0.00,8.00,2020-09-04,2020-09-04
22,Zelger Samuel,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-05-08,ZLGSML18E08F132P,Ambach Maria,Eppan,2020-08-21,2020-08-21,1,7.00,0.00,0,0.00,7.00,2020-08-21,2020-08-21
24,Zelger Samuel,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-05-08,ZLGSML18E08F132P,Ambach Maria,Eppan,2020-05-18,2020-06-18,32,74.75,0.00,0,0.00,74.75,2020-05-18,2020-06-18


In [37]:
df["Comune di residenza assistente domiciliare all'infanzia"]

8         Bozen
9     Kurtatsch
10      Kaltern
11        Eppan
12        Eppan
13     Margreid
14    Kurtatsch
15    Kurtatsch
16        Eppan
17        Eppan
18     Margreid
19    Kurtatsch
20     Margreid
21        Eppan
22        Eppan
23     Margreid
24        Eppan
Name: Comune di residenza assistente domiciliare all'infanzia, dtype: object

In [55]:
df.insert(12,"massimo543",0)

ValueError: cannot insert massimo543, already exists

In [56]:
df

,Cognome e nome bambino,Ente,Comune,Data di nascita,Codice fiscale,Assistente domiciliare all'infanzia,Comune di residenza assistente domiciliare all'infanzia,Data inizio contratto (o data inizio assistenza se diversa),Data fine contratto\n(o data fine assistenza se diversa) *,GiorniAssistenzaAnnoRiferimento,massimo543,Ore di assistenza \n ai sensi della delibera\nn. 666/2019,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020,Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo),Ore totali rendicontate per il 2020,inizioNorm,fineNorm
8,Chirone Riccardo,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-10-09,CHRRCR18R09E986O,Salvadori Paola,Bozen,2019-11-19,2020-12-31,366,0,1308.75,89.50,0,-340.00,1058.25,2020-01-01,2020-12-31
9,Frei Juri,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-05-29,FREJRU18E29A952R,Peer Petra,Kurtatsch,2020-03-10,2020-05-05,57,0,0.00,90.00,0,0.00,90.00,2020-03-10,2020-05-05
10,Kajana Mia,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-08-03,KJNMIA18M43A952A,Scarduelli Sonia,Kaltern,2020-05-18,2020-12-31,228,0,559.50,24.00,0,-1.75,581.75,2020-05-18,2020-12-31
11,Kajana Mia,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-08-03,KJNMIA18M43A952A,Ambach Maria,Eppan,2019-08-06,2020-05-04,125,0,99.50,144.75,0,66.25,310.50,2020-01-01,2020-05-04
12,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Ambach Maria,Eppan,2020-08-17,2020-08-28,12,0,71.50,0.00,0,0.00,71.50,2020-08-17,2020-08-28
13,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Huber Melanie,Margreid,2020-11-25,2020-11-27,3,0,26.50,0.00,0,0.00,26.50,2020-11-25,2020-11-27
14,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Peer Petra,Kurtatsch,2020-01-02,2020-05-06,126,0,189.75,270.50,0,332.50,792.75,2020-01-02,2020-05-06
15,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Peer Petra,Kurtatsch,2020-07-20,2020-12-31,165,0,804.00,10.25,0,0.00,814.25,2020-07-20,2020-12-31
16,Von Call Dylan,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-19,VNCDLN19B19A952G,Ambach Maria,Eppan,2020-07-20,2020-11-23,127,0,231.50,20.75,0,0.00,252.25,2020-07-20,2020-11-23
17,Von Call Dylan,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-19,VNCDLN19B19A952G,Ambach Maria,Eppan,2019-11-25,2020-05-06,127,0,173.50,279.50,0,364.50,817.50,2020-01-01,2020-05-06


In [57]:
df.columns

Index(['Cognome e nome bambino', 'Ente', 'Comune', 'Data di nascita',
       'Codice fiscale', 'Assistente domiciliare all'infanzia',
       'Comune di residenza assistente domiciliare all'infanzia',
       'Data inizio contratto (o data inizio assistenza se diversa)',
       'Data fine contratto\n(o data fine assistenza se diversa) *',
       'GiorniAssistenzaAnnoRiferimento', 'massimo543',
       'Ore di assistenza \n ai sensi della delibera\nn. 666/2019',
       'Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020',
       'Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020',
       'Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo)',
       'Ore totali rendicontate per il 2020', 'inizioNorm', 'fineNorm'],
      dtype='object')

In [61]:
df["pippo"]= 0

In [62]:
df.columns

Index(['Cognome e nome bambino', 'Ente', 'Comune', 'Data di nascita',
       'Codice fiscale', 'Assistente domiciliare all'infanzia',
       'Comune di residenza assistente domiciliare all'infanzia',
       'Data inizio contratto (o data inizio assistenza se diversa)',
       'Data fine contratto\n(o data fine assistenza se diversa) *',
       'GiorniAssistenzaAnnoRiferimento', 'massimo543',
       'Ore di assistenza \n ai sensi della delibera\nn. 666/2019',
       'Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020',
       'Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020',
       'Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo)',
       'Ore totali rendicontate per il 2020', 'inizioNorm', 'fineNorm',
       'pippo'],
      dtype='object')

In [63]:
    cond1 = df[
        "Data inizio contratto (o data inizio assistenza se diversa)"
    ] <= pd.to_datetime("05.03.2020", format="%d.%m.%Y")
    cond1b = df[
        "Data fine contratto\n(o data fine assistenza se diversa) *"
    ] > pd.to_datetime("17.05.2020", format="%d.%m.%Y")

In [86]:
df.loc[cond1 & cond1b,"pippo"] = 99

In [88]:
df[cond1 & cond1b]

,Cognome e nome bambino,Ente,Comune,Data di nascita,Codice fiscale,Assistente domiciliare all'infanzia,Comune di residenza assistente domiciliare all'infanzia,Data inizio contratto (o data inizio assistenza se diversa),Data fine contratto\n(o data fine assistenza se diversa) *,GiorniAssistenzaAnnoRiferimento,massimo543,Ore di assistenza \n ai sensi della delibera\nn. 666/2019,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020,Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo),Ore totali rendicontate per il 2020,inizioNorm,fineNorm,pippo
8,Chirone Riccardo,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-10-09,CHRRCR18R09E986O,Salvadori Paola,Bozen,2019-11-19,2020-12-31,366,0,1308.75,89.5,0,-340.0,1058.25,2020-01-01,2020-12-31,99


In [97]:
df["Comune di residenza assistente domiciliare all'infanzia"].str.lower().str.contains("bozen")

8      True
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
Name: Comune di residenza assistente domiciliare all'infanzia, dtype: bool

In [100]:
df.insert(13, "massimo545", 0).astype(int)

AttributeError: 'NoneType' object has no attribute 'astype'

In [10]:
df

,Cognome e nome bambino,Ente,Comune,Data di nascita,Codice fiscale,Assistente domiciliare all'infanzia,Comune di residenza assistente domiciliare all'infanzia,Data inizio contratto (o data inizio assistenza se diversa),Data fine contratto\n(o data fine assistenza se diversa) *,GiorniAssistenzaAnnoRiferimento,Ore di assistenza \n ai sensi della delibera\nn. 666/2019,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020,Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo),Ore totali rendicontate per il 2020,inizioNorm,fineNorm
8,Chirone Riccardo,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-10-09,CHRRCR18R09E986O,Salvadori Paola,Bozen,2019-11-19,2020-12-31,366,1308.75,89.50,0,-340.00,1058.25,2020-01-01,2020-12-31
9,Frei Juri,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-05-29,FREJRU18E29A952R,Peer Petra,Kurtatsch,2020-03-10,2020-05-05,57,0.00,90.00,0,0.00,90.00,2020-03-10,2020-05-05
10,Kajana Mia,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-08-03,KJNMIA18M43A952A,Scarduelli Sonia,Kaltern,2020-05-18,2020-12-31,228,559.50,24.00,0,-1.75,581.75,2020-05-18,2020-12-31
11,Kajana Mia,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-08-03,KJNMIA18M43A952A,Ambach Maria,Eppan,2019-08-06,2020-05-04,125,99.50,144.75,0,66.25,310.50,2020-01-01,2020-05-04
12,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Ambach Maria,Eppan,2020-08-17,2020-08-28,12,71.50,0.00,0,0.00,71.50,2020-08-17,2020-08-28
13,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Huber Melanie,Margreid,2020-11-25,2020-11-27,3,26.50,0.00,0,0.00,26.50,2020-11-25,2020-11-27
14,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Peer Petra,Kurtatsch,2020-01-02,2020-05-06,126,189.75,270.50,0,332.50,792.75,2020-01-02,2020-05-06
15,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Peer Petra,Kurtatsch,2020-07-20,2020-12-31,165,804.00,10.25,0,0.00,814.25,2020-07-20,2020-12-31
16,Von Call Dylan,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-19,VNCDLN19B19A952G,Ambach Maria,Eppan,2020-07-20,2020-11-23,127,231.50,20.75,0,0.00,252.25,2020-07-20,2020-11-23
17,Von Call Dylan,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-19,VNCDLN19B19A952G,Ambach Maria,Eppan,2019-11-25,2020-05-06,127,173.50,279.50,0,364.50,817.50,2020-01-01,2020-05-06


In [14]:
datetime()

NameError: name 'datetime' is not defined

In [17]:
from datetime import date

In [22]:
date.today().strftime("%d/%m/%Y")

'10/06/2022'

In [28]:
df = df.append({"Comune di residenza assistente domiciliare all'infanzia":"file creato il","Data inizio contratto (o data inizio assistenza se diversa)":date.today().strftime("%d/%m/%Y")}, ignore_index=True)

In [38]:
df = df.append({}, ignore_index=True )

In [39]:
df

,Cognome e nome bambino,Ente,Comune,Data di nascita,Codice fiscale,Assistente domiciliare all'infanzia,Comune di residenza assistente domiciliare all'infanzia,Data inizio contratto (o data inizio assistenza se diversa),Data fine contratto\n(o data fine assistenza se diversa) *,GiorniAssistenzaAnnoRiferimento,Ore di assistenza \n ai sensi della delibera\nn. 666/2019,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020,Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo),Ore totali rendicontate per il 2020,inizioNorm,fineNorm
0,Chirone Riccardo,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-10-09,CHRRCR18R09E986O,Salvadori Paola,Bozen,2019-11-19 00:00:00,2020-12-31,366.0,1308.75,89.50,0.0,-340.00,1058.25,2020-01-01,2020-12-31
1,Frei Juri,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-05-29,FREJRU18E29A952R,Peer Petra,Kurtatsch,2020-03-10 00:00:00,2020-05-05,57.0,0.00,90.00,0.0,0.00,90.00,2020-03-10,2020-05-05
2,Kajana Mia,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-08-03,KJNMIA18M43A952A,Scarduelli Sonia,Kaltern,2020-05-18 00:00:00,2020-12-31,228.0,559.50,24.00,0.0,-1.75,581.75,2020-05-18,2020-12-31
3,Kajana Mia,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-08-03,KJNMIA18M43A952A,Ambach Maria,Eppan,2019-08-06 00:00:00,2020-05-04,125.0,99.50,144.75,0.0,66.25,310.50,2020-01-01,2020-05-04
4,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Ambach Maria,Eppan,2020-08-17 00:00:00,2020-08-28,12.0,71.50,0.00,0.0,0.00,71.50,2020-08-17,2020-08-28
5,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Huber Melanie,Margreid,2020-11-25 00:00:00,2020-11-27,3.0,26.50,0.00,0.0,0.00,26.50,2020-11-25,2020-11-27
6,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Peer Petra,Kurtatsch,2020-01-02 00:00:00,2020-05-06,126.0,189.75,270.50,0.0,332.50,792.75,2020-01-02,2020-05-06
7,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Peer Petra,Kurtatsch,2020-07-20 00:00:00,2020-12-31,165.0,804.00,10.25,0.0,0.00,814.25,2020-07-20,2020-12-31
8,Von Call Dylan,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-19,VNCDLN19B19A952G,Ambach Maria,Eppan,2020-07-20 00:00:00,2020-11-23,127.0,231.50,20.75,0.0,0.00,252.25,2020-07-20,2020-11-23
9,Von Call Dylan,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-19,VNCDLN19B19A952G,Ambach Maria,Eppan,2019-11-25 00:00:00,2020-05-06,127.0,173.50,279.50,0.0,364.50,817.50,2020-01-01,2020-05-06


In [40]:
df

,Cognome e nome bambino,Ente,Comune,Data di nascita,Codice fiscale,Assistente domiciliare all'infanzia,Comune di residenza assistente domiciliare all'infanzia,Data inizio contratto (o data inizio assistenza se diversa),Data fine contratto\n(o data fine assistenza se diversa) *,GiorniAssistenzaAnnoRiferimento,Ore di assistenza \n ai sensi della delibera\nn. 666/2019,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 543_1025/2020,Ore contrattualizzate non erogate\nai sensi della delibera\nn. 733/2020,Ore contrattualizzate non erogate\nnella fase 2 (finanziamento compensativo),Ore totali rendicontate per il 2020,inizioNorm,fineNorm
0,Chirone Riccardo,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-10-09,CHRRCR18R09E986O,Salvadori Paola,Bozen,2019-11-19 00:00:00,2020-12-31,366.0,1308.75,89.50,0.0,-340.00,1058.25,2020-01-01,2020-12-31
1,Frei Juri,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-05-29,FREJRU18E29A952R,Peer Petra,Kurtatsch,2020-03-10 00:00:00,2020-05-05,57.0,0.00,90.00,0.0,0.00,90.00,2020-03-10,2020-05-05
2,Kajana Mia,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-08-03,KJNMIA18M43A952A,Scarduelli Sonia,Kaltern,2020-05-18 00:00:00,2020-12-31,228.0,559.50,24.00,0.0,-1.75,581.75,2020-05-18,2020-12-31
3,Kajana Mia,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2018-08-03,KJNMIA18M43A952A,Ambach Maria,Eppan,2019-08-06 00:00:00,2020-05-04,125.0,99.50,144.75,0.0,66.25,310.50,2020-01-01,2020-05-04
4,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Ambach Maria,Eppan,2020-08-17 00:00:00,2020-08-28,12.0,71.50,0.00,0.0,0.00,71.50,2020-08-17,2020-08-28
5,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Huber Melanie,Margreid,2020-11-25 00:00:00,2020-11-27,3.0,26.50,0.00,0.0,0.00,26.50,2020-11-25,2020-11-27
6,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Peer Petra,Kurtatsch,2020-01-02 00:00:00,2020-05-06,126.0,189.75,270.50,0.0,332.50,792.75,2020-01-02,2020-05-06
7,Tomoiaga Ema Miriam,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-01,TMGMRM19B41A952N,Peer Petra,Kurtatsch,2020-07-20 00:00:00,2020-12-31,165.0,804.00,10.25,0.0,0.00,814.25,2020-07-20,2020-12-31
8,Von Call Dylan,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-19,VNCDLN19B19A952G,Ambach Maria,Eppan,2020-07-20 00:00:00,2020-11-23,127.0,231.50,20.75,0.0,0.00,252.25,2020-07-20,2020-11-23
9,Von Call Dylan,Sozialgenossenschaft Mit Bäuerinnen Lernen-Wac...,Tramin,2019-02-19,VNCDLN19B19A952G,Ambach Maria,Eppan,2019-11-25 00:00:00,2020-05-06,127.0,173.50,279.50,0.0,364.50,817.50,2020-01-01,2020-05-06
